# Unsealing

Vault offers a variety of seals to encrypt its backend storage. The standard seal is the Shamir seal based on [Shamir's secret sharing](https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing). However Vault also offers other seal mechanisms that rely on external systems to provide cryptographic capabilities to enable the decription of the backend.

## Shamir Seals

Let us first look at Shamir seals. They rely on key shares that need to be combined in order to create the final key enabling the decription of the storage backend. It should be noted that the Shamir secret itself does not encrypt the backend directly, but rather encrypts the master encryption key of the backend, such that the encrypted version of the master encryption key can be stored on disk.

![](assets/img/shamir-keys.png)

First, let us check the Vault configuration. The Shamir seal is the default, and is configured by the **absence** of any `seal` stanza in the configuration:

In [ ]:
cat ./assets/shamir-config.hcl

Now let us start a Vault instance with this configuration:

In [ ]:
nohup bash -c '
  vault server -config ./assets/shamir-config.hcl
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Checking Seal Status

The Vault at this point is not initialised. We will first need to initialize its storage backend. When initializing the storage, we can provide the number of key shares we would like to be generated for the Shamir seal. Moreover, we can provide the threshold of shares required to generate the key. Only a subset the size of the threshold will be required to unseal the Vault. This enables the distribution of individual key shares across various people, while allowing a subset of these people to then unseal the Vault whenever necessary. A single person with a single key share can however never perform any action by themselves (unless the threshold is set to 1). This is important because this threshold of key shares is also required to generate root tokens, which a single person should never be able to do by themselves.

Below we will first check the status of the Vault and validate that it is not initialized. Then, use the `vault operator init` command with the `-key-shares` and `-key-threshold` flags to initialise the storage.

We pipe the output of that command to a file (`/tmp/vault-unseal.log`) as its output is critical to retain.

Thus:

1. Check the status of the vault using `vault status`.
2. Initialise the storage backend with a **threshold of 2** and **3 key shares** using the `vault operator init` command.

In [ ]:
# check status and see that the `Initialized` field is `false`
export VAULT_ADDR="http://localhost:8200"
vault status

In [ ]:
# initialise the Vault storage backend (and thus the seal)
vault operator init <fill in here> | tee /tmp/vault-unseal.log

### Unsealing the Vault

Now you can use the `vault operator unseal` command to unseal the Vault. You will need to repeat this with 2 different key shares in order to unseal the Vault:

In [ ]:
vault operator unseal "<your key share>"

In [ ]:
vault operator unseal "<your key share>"

### Checking the Status Again

Your last unseal command should have returned `false` in the `Sealed` field. This means that Vault is now unsealed and thus ready to use. You can therefore try to login and perform actions against the Vault. Let us try this:

1. Login to the Vault using the `vault login` command with the root token returned from the `vault operator init` command above.

In [ ]:
vault login "<your root token here>"

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault/vault.db
rm -r /tmp/vault/raft/
rm /tmp/vault.log
rm /tmp/vault.pid
rm /tmp/vault-unseal.log